In [2]:
import tensorflow as tf
from tensorflow import keras
import os
import string
import chromadb
import warnings
warnings.filterwarnings('ignore')
from transformers import pipeline
from llama_cpp import Llama


# onderstaande koste mij ongeveer een halfuur
# pip install llama_cpp-python


In [3]:
documents = []
metadata = []
ids = []

with open('data/spells.txt', 'r', encoding='utf-8') as spells:
    for line in spells:
        line=line.split(" - ") #spells are separated by their description by a " - "
        metadata.append({'spell': line[0]}) #use the spell name as metadata. ChromaDB requires a dictionary here
        documents.append(line[1]) #use the description as documents
        ids.append(line[0]) #we also need an ID so use the spell name again.
print(ids)

['Aberto', 'Accio', 'Aguamenti', 'Alohomora', 'Anapneo', 'Aparecium', 'Apparate', 'Ascendio', 'Avada Kedavra', 'Avis', 'Bat-Bogey Hex', 'Bombardo', 'Brackium Emendo', 'Capacious Extremis', 'Confundo', 'Conjunctivitis Curse', 'Crinus Muto', 'Crucio', 'Diffindo', 'Disillusionment Charm', 'Disapparate', 'Engorgio', 'Episkey', 'Expecto patronum', 'Erecto', 'Evanesco', 'Expelliarmus', 'Ferula', 'Fidelius Charm', 'Fiendfyre Curse', 'Finite Incantatem', 'Furnunculus Curse', 'Geminio', 'Glisseo', 'Homenum Revelio', 'Homonculus Charm', 'Immobulus', 'Impedimenta', 'Incarcerous', 'Imperio', 'Impervius', 'Incendio', 'Langlock', 'Legilimens', 'Levicorpus', 'Locomotor Mortis', 'Lumos', 'Morsmordre', 'Mucus Ad Nauseam', 'Muffliato', 'Nox', 'Obliviate', 'Obscuro', 'Oculus Reparo', 'Oppugno', 'Petrificus Totalus', 'Periculum', 'Piertotum Locomotor', 'Protean Charm', 'Protego', 'Reducto', 'Reducio', 'Renneverate', 'Reparifors', 'Reparo', 'Rictusempra', 'Riddikulus', 'Scourgify', 'Sectumsempra', 'Serpens

In [4]:
client = chromadb.Client()

# create or open collection named 'spells' 
spells_collection = client.get_or_create_collection("spells")

# add spells to the collection, ignored for ids that already exist 
spells_collection.add(
    documents=documents,
    metadatas=metadata,
    ids=ids
)

In [5]:
# En dit koste mij ook weer een halfuur

llm = Llama.from_pretrained(
	repo_id="BramVanroy/GEITje-7B-ultra-GGUF",
	filename="geitje-7b-ultra-q8_0.gguf",
)

llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": "What is the capital of France?"
		}
	]
)

llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from C:\Users\raymo\.cache\huggingface\hub\models--BramVanroy--GEITje-7B-ultra-GGUF\snapshots\00da1c06460774d6982d608e495dac102ead3ec1\.\geitje-7b-ultra-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = geitje-7b-ultra
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 32768
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:            

{'id': 'chatcmpl-c07ec2fd-ca7c-4280-8699-762cbf767b93',
 'object': 'chat.completion',
 'created': 1728973138,
 'model': 'C:\\Users\\raymo\\.cache\\huggingface\\hub\\models--BramVanroy--GEITje-7B-ultra-GGUF\\snapshots\\00da1c06460774d6982d608e495dac102ead3ec1\\.\\geitje-7b-ultra-q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Het capital van Frankrijk is Parijs.'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 23, 'completion_tokens': 11, 'total_tokens': 34}}

In [9]:
user_input = input("Ask your question in English:")
results = spells_collection.query(
    query_texts=[user_input],
    n_results=1
)
results

{'ids': [['Bat-Bogey Hex']],
 'distances': [[0.7008856534957886]],
 'metadatas': [[{'spell': 'Bat-Bogey Hex'}]],
 'embeddings': None,
 'documents': [['Verandert het snot van het doelwit in vleermuizen\n']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [10]:
result=results['metadatas'][0][0]['spell']+" - " + results['documents'][0][0][:-1]
result

'Bat-Bogey Hex - Verandert het snot van het doelwit in vleermuizen'

In [11]:
prompt = f"""
voorgestelde spreuk: {result}
User input: {user_input}
stel een spreuk voor op basis van de voorgestelde spreuk en de user input.
"""
print("Prompt: ", prompt)
inference = llm(prompt, max_tokens=100)
generated_text = inference['choices'][0]['text']
print("Inference: ", generated_text)

Prompt:  
voorgestelde spreuk: Bat-Bogey Hex - Verandert het snot van het doelwit in vleermuizen
User input: hoe laat ik iets vliegen?
stel een spreuk voor op basis van de voorgestelde spreuk en de user input.



Llama.generate: 11 prefix-match hit, remaining 66 prompt tokens to eval
llama_perf_context_print:        load time =   17283.83 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    66 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    99 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   75352.89 ms /   165 tokens


Inference:  
In de spreuk zou je elementen van de verandering in snot naar vleermuizen kunnen verwerken, als een metafoor voor transformatie, en tegelijkertijd een verwijzing naar de tijd om iets te laten vliegen. Hier is een aangepaste spreuk:

"Bat-Bogey Hex - Vleermuizen van snot, stijg op!
